In [1]:
!export LD_LIBRARY_PATH=/usr/local/tensorrt/lib:$LD_LIBRARY_PATH

In [2]:
!source ~/.bashrc  # or source ~/.zshrc

In [3]:
!dpkg -l | grep nvinfer

In [4]:
!dpkg -l | grep tensorrt

In [5]:
import tensorflow as tf
import argparse
import pandas as pd
import os
import sys
import argparse
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
print("TensorFlow version:", tf.__version__)

2025-02-26 12:21:58.104669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 12:21:59.314614: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-02-26 12:21:59.314729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

TensorFlow version: 2.11.0


# **Creates a new directory called trainerfinal**

In [6]:
%%bash
mkdir trainerfinal
touch trainerfinal/__init__.py

# **Download data and run the model and deploy**

In [7]:
def load_data():
    data_train = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_train.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    data_test = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_test.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    return data_train, data_test

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(9,)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mape'])
    return model

def train_and_evaluate(output_dir):
    data_train, data_test = load_data()
    FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO"]
    LABEL = "MEDV"
    
    X_train, y_train = data_train[FEATURES].values, data_train[LABEL].values
    X_test, y_test = data_test[FEATURES].values, data_test[LABEL].values
    
    model = build_model()
    model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_test, y_test))
    model.save(os.path.join(output_dir, "housing_model.keras"))
    print("Model saved to", output_dir)

def main(output_dir, deploy, bucket_name=None, job_name=None):
    os.makedirs(output_dir, exist_ok=True)
    train_and_evaluate(output_dir)
    
    if deploy == "cloud":
        print("Uploading model to Cloud Storage...")
        os.system(f"gsutil cp -r {output_dir} gs://{bucket_name}/{job_name}/")
        print("Model uploaded to gs://{}/{}".format(bucket_name, job_name))

In [8]:
if __name__ == "__main__":
    if "ipykernel_launcher" in sys.argv[0]:  # Detects Jupyter Notebook
        args = argparse.Namespace(
            output_dir="./model_output",
            deploy="local",  # Change to "cloud" if deploying to GCP
            bucket_name="your-bucket-name",  # Needed for cloud
            job_name="housing_model_training"  # Needed for cloud
        )
    else:
        parser = argparse.ArgumentParser()
        parser.add_argument("--output_dir", required=True, help="Path to save the model")
        parser.add_argument("--deploy", required=True, choices=["local", "cloud"], help="Deployment mode")
        parser.add_argument("--bucket_name", help="Google Cloud Storage bucket name")
        parser.add_argument("--job_name", help="Google Cloud AI Platform job name")
        args = parser.parse_args()

    main(args.output_dir, args.deploy, args.bucket_name, args.job_name)


Epoch 1/30


2025-02-26 12:23:24.983208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-02-26 12:23:24.983262: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-26 12:23:24.983287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20250226-162453): /proc/driver/nvidia/version does not exist
2025-02-26 12:23:24.983616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable the

41/41 [==============================] - 1s 8ms/step - loss: 668966976.0000 - mape: 99.8736 - val_loss: 280387680.0000 - val_mape: 99.2401
Epoch 2/30
41/41 [==============================] - 0s 3ms/step - loss: 663219392.0000 - mape: 99.2648 - val_loss: 267507776.0000 - val_mape: 96.2187
Epoch 3/30
41/41 [==============================] - 0s 3ms/step - loss: 638218880.0000 - mape: 96.4859 - val_loss: 214466848.0000 - val_mape: 82.7446
Epoch 4/30
41/41 [==============================] - 0s 3ms/step - loss: 544524672.0000 - mape: 85.0209 - val_loss: 79142272.0000 - val_mape: 40.5269
Epoch 5/30
41/41 [==============================] - 0s 3ms/step - loss: 324647008.0000 - mape: 60.5177 - val_loss: 192129952.0000 - val_mape: 102.8927
Epoch 6/30
41/41 [==============================] - 0s 3ms/step - loss: 189517984.0000 - mape: 50.7541 - val_loss: 494105152.0000 - val_mape: 163.6092
Epoch 7/30
41/41 [==============================] - 0s 3ms/step - loss: 185496112.0000 - mape: 52.0626 - val_l

## **Run the model on local device**

In [9]:
main(output_dir="./model_output", deploy="local")


Epoch 1/30
41/41 [==============================] - 1s 6ms/step - loss: 669846976.0000 - mape: 99.9785 - val_loss: 282265248.0000 - val_mape: 99.6716
Epoch 2/30
41/41 [==============================] - 0s 3ms/step - loss: 666226496.0000 - mape: 99.5809 - val_loss: 274760288.0000 - val_mape: 97.9391
Epoch 3/30
41/41 [==============================] - 0s 3ms/step - loss: 651948992.0000 - mape: 97.9113 - val_loss: 247265584.0000 - val_mape: 91.3250
Epoch 4/30
41/41 [==============================] - 0s 3ms/step - loss: 605271872.0000 - mape: 92.4944 - val_loss: 172100432.0000 - val_mape: 70.5145
Epoch 5/30
41/41 [==============================] - 0s 3ms/step - loss: 488049664.0000 - mape: 78.5657 - val_loss: 57205148.0000 - val_mape: 34.9556
Epoch 6/30
41/41 [==============================] - 0s 3ms/step - loss: 299205920.0000 - mape: 57.0786 - val_loss: 148544384.0000 - val_mape: 90.3468
Epoch 7/30
41/41 [==============================] - 0s 3ms/step - loss: 184617008.0000 - mape: 48.867

## **Bucket_name = Create a bucket in the Googlecloud with any name(msds462-dl) and give that name here**

In [10]:
main(output_dir="./model_output", deploy="cloud", bucket_name="msds462-dl", job_name="housing_model_training")

Epoch 1/30
41/41 [==============================] - 1s 7ms/step - loss: 670193024.0000 - mape: 100.0175 - val_loss: 283498144.0000 - val_mape: 99.9524
Epoch 2/30
41/41 [==============================] - 0s 3ms/step - loss: 669477568.0000 - mape: 99.9368 - val_loss: 282360928.0000 - val_mape: 99.6924
Epoch 3/30
41/41 [==============================] - 0s 3ms/step - loss: 667153664.0000 - mape: 99.6782 - val_loss: 277103936.0000 - val_mape: 98.4806
Epoch 4/30
41/41 [==============================] - 0s 3ms/step - loss: 657657984.0000 - mape: 98.6402 - val_loss: 257810880.0000 - val_mape: 93.9033
Epoch 5/30
41/41 [==============================] - 0s 3ms/step - loss: 625206912.0000 - mape: 94.7925 - val_loss: 198352208.0000 - val_mape: 78.3102
Epoch 6/30
41/41 [==============================] - 0s 3ms/step - loss: 522709728.0000 - mape: 82.7966 - val_loss: 67860672.0000 - val_mape: 37.3557
Epoch 7/30
41/41 [==============================] - 0s 3ms/step - loss: 306320480.0000 - mape: 59.15

Copying file://./model_output/housing_model.keras [Content-Type=application/octet-stream]...
/ [1 files][ 59.4 KiB/ 59.4 KiB]                                                
Operation completed over 1 objects/59.4 KiB.                                     


Model uploaded to gs://msds462-dl/housing_model_training


**Now Check in the bucket if the model has been created in the Google cloud with housing_model.keras**

## Suggested Exploration

### (a) Change the number of units of the DNNRegressor used to train the model and observe the impact on model accuracy

In [11]:
def load_data():
    data_train = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_train.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    data_test = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_test.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    return data_train, data_test

def build_model(units_per_layer):
    """
    Build a model with customizable units per layer.
    :param units_per_layer: List of integers representing the number of units per layer.
    :return: Keras model
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(9,)))
    
    # Add layers based on the units_per_layer list
    for units in units_per_layer:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
    
    # Output layer
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mape'])
    return model

def train_and_evaluate(output_dir, units_per_layer):
    data_train, data_test = load_data()
    FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO"]
    LABEL = "MEDV"
    
    X_train, y_train = data_train[FEATURES].values, data_train[LABEL].values
    X_test, y_test = data_test[FEATURES].values, data_test[LABEL].values
    
    model = build_model(units_per_layer)
    model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_test, y_test))
    model.save(os.path.join(output_dir, "housing_model1.keras"))
    print("Model saved to", output_dir)

def main(output_dir, deploy, units_per_layer, bucket_name=None, job_name=None):
    os.makedirs(output_dir, exist_ok=True)
    train_and_evaluate(output_dir, units_per_layer)
    
    if deploy == "cloud":
        print("Uploading model to Cloud Storage...")
        os.system(f"gsutil cp -r {output_dir} gs://{bucket_name}/{job_name}/")
        print("Model uploaded to gs://{}/{}".format(bucket_name, job_name))


In [12]:
# Directly set parameters when running in Jupyter (bypassing argparse)
if __name__ == "__main__":
    # Manually set the arguments for Jupyter Notebook execution
    args = argparse.Namespace(
        output_dir="./model_output",
        deploy="local",  # Change to "cloud" if deploying to GCP
        bucket_name="your-bucket-name",  # Needed for cloud
        job_name="housing_model_training",  # Needed for cloud
        units_per_layer=[5, 10, 15]  # Default unit configuration for layers
    )
    
    # Now call the main function with all the arguments, including units_per_layer
    main(args.output_dir, args.deploy, args.units_per_layer, args.bucket_name, args.job_name)

Epoch 1/30
41/41 [==============================] - 1s 6ms/step - loss: 670018560.0000 - mape: 99.9998 - val_loss: 283701024.0000 - val_mape: 99.9992
Epoch 2/30
41/41 [==============================] - 0s 3ms/step - loss: 670010880.0000 - mape: 99.9991 - val_loss: 283696416.0000 - val_mape: 99.9982
Epoch 3/30
41/41 [==============================] - 0s 2ms/step - loss: 669983168.0000 - mape: 99.9974 - val_loss: 283689856.0000 - val_mape: 99.9967
Epoch 4/30
41/41 [==============================] - 0s 3ms/step - loss: 669814400.0000 - mape: 99.9871 - val_loss: 283676544.0000 - val_mape: 99.9938
Epoch 5/30
41/41 [==============================] - 0s 3ms/step - loss: 668334528.0000 - mape: 99.8616 - val_loss: 279912736.0000 - val_mape: 99.1689
Epoch 6/30
41/41 [==============================] - 0s 3ms/step - loss: 658473344.0000 - mape: 98.8144 - val_loss: 263293648.0000 - val_mape: 95.2930
Epoch 7/30
41/41 [==============================] - 0s 3ms/step - loss: 633387840.0000 - mape: 96.10

### Key Observations:
1. Slower Convergence: The new model takes longer to converge, with a slower reduction in loss and MAPE. This could indicate that increasing the units in the regressor has led to a more complex model, which is more difficult to train.
2. Overfitting: The new model shows more overfitting compared to the initial model. Even though the training loss is decreasing, the validation loss is not improving significantly and in some cases, it even increases. Additionally, the validation MAPE is much higher and less stable.
3. Poorer Generalization: The validation performance deteriorates over epochs, indicating that the model with the new configuration struggles to generalize from training to validation data. In contrast, the initial model had a better balance between training and validation performance, with validation MAPE staying relatively lower compared to the new model.

### (b) Observe how the model training is being scaled across single/several VMs

In [13]:
def load_data():
    data_train = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_train.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    data_test = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_test.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    return data_train, data_test

def build_model(units_per_layer):
    """
    Build a model with customizable units per layer.
    :param units_per_layer: List of integers representing the number of units per layer.
    :return: Keras model
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(9,)))
    
    # Add layers based on the units_per_layer list
    for units in units_per_layer:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
    
    # Output layer
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mape'])
    return model

def train_and_evaluate(output_dir, units_per_layer):
    data_train, data_test = load_data()
    FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO"]
    LABEL = "MEDV"
    
    X_train, y_train = data_train[FEATURES].values, data_train[LABEL].values
    X_test, y_test = data_test[FEATURES].values, data_test[LABEL].values
    
    model = build_model(units_per_layer)
    
    # Set up TensorBoard callback
    log_dir = os.path.join(output_dir, "logs")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Train the model with the TensorBoard callback
    model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    
    model.save(os.path.join(output_dir, "housing_model1.keras"))
    print("Model saved to", output_dir)

def main(output_dir, deploy, units_per_layer, bucket_name=None, job_name=None):
    os.makedirs(output_dir, exist_ok=True)
    train_and_evaluate(output_dir, units_per_layer)
    
    if deploy == "cloud":
        print("Uploading model to Cloud Storage...")
        os.system(f"gsutil cp -r {output_dir} gs://{bucket_name}/{job_name}/")
        print("Model uploaded to gs://{}/{}".format(bucket_name, job_name))


# Directly set parameters when running in Jupyter (bypassing argparse)
if __name__ == "__main__":
    # Manually set the arguments for Jupyter Notebook execution
    args = argparse.Namespace(
        output_dir="./model_output",
        deploy="local",  # Change to "cloud" if deploying to GCP
        bucket_name="your-bucket-name",  # Needed for cloud
        job_name="housing_model_training",  # Needed for cloud
        units_per_layer=[5, 10, 15]  # Default unit configuration for layers
    )
    
    # Now call the main function with all the arguments, including units_per_layer
    main(args.output_dir, args.deploy, args.units_per_layer, args.bucket_name, args.job_name)

    # Run TensorBoard in the notebook after training
    %load_ext tensorboard
    %tensorboard --logdir=./model_output/logs


Epoch 1/30
41/41 [==============================] - 1s 9ms/step - loss: 667051584.0000 - mape: 99.6713 - val_loss: 278449920.0000 - val_mape: 98.7889
Epoch 2/30
41/41 [==============================] - 0s 4ms/step - loss: 662877952.0000 - mape: 99.2185 - val_loss: 271595072.0000 - val_mape: 97.1878
Epoch 3/30
41/41 [==============================] - 0s 4ms/step - loss: 653668224.0000 - mape: 98.1685 - val_loss: 256697104.0000 - val_mape: 93.6229
Epoch 4/30
41/41 [==============================] - 0s 4ms/step - loss: 633861184.0000 - mape: 95.9369 - val_loss: 227260208.0000 - val_mape: 86.1904
Epoch 5/30
41/41 [==============================] - 0s 4ms/step - loss: 596294272.0000 - mape: 91.2560 - val_loss: 177726240.0000 - val_mape: 72.1834
Epoch 6/30
41/41 [==============================] - 0s 4ms/step - loss: 533358720.0000 - mape: 83.6969 - val_loss: 112177304.0000 - val_mape: 50.4526
Epoch 7/30
41/41 [==============================] - 0s 4ms/step - loss: 444299744.0000 - mape: 73.20

### Use MultiWorkerMirroredStrategy for Distributed Training

In [14]:
def load_data():
    data_train = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_train.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    data_test = pd.read_csv(
        'https://storage.googleapis.com/spls/gsp418/housing_test.csv',
        names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "MEDV"])
    return data_train, data_test

def build_model(units_per_layer):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(9,)))
    
    for units in units_per_layer:
        model.add(layers.Dense(units, activation='relu'))
    
    model.add(layers.Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mape'])
    return model

def train_and_evaluate(output_dir, units_per_layer):
    data_train, data_test = load_data()
    FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO"]
    LABEL = "MEDV"
    
    X_train, y_train = data_train[FEATURES].values, data_train[LABEL].values
    X_test, y_test = data_test[FEATURES].values, data_test[LABEL].values

    # Using MultiWorkerMirroredStrategy for scaling across multiple workers
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
    
    with strategy.scope():
        model = build_model(units_per_layer)
        
        # Set up TensorBoard callback
        log_dir = os.path.join(output_dir, "logs")
        tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

        # Train the model with the TensorBoard callback
        model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    
    model.save(os.path.join(output_dir, "housing_model1.keras"))
    print("Model saved to", output_dir)

def main(output_dir, deploy, units_per_layer, bucket_name=None, job_name=None):
    os.makedirs(output_dir, exist_ok=True)
    train_and_evaluate(output_dir, units_per_layer)
    
    if deploy == "cloud":
        print("Uploading model to Cloud Storage...")
        os.system(f"gsutil cp -r {output_dir} gs://{bucket_name}/{job_name}/")
        print("Model uploaded to gs://{}/{}".format(bucket_name, job_name))


# Directly set parameters when running in Jupyter (bypassing argparse)
if __name__ == "__main__":
    # Manually set the arguments for Jupyter Notebook execution
    args = argparse.Namespace(
        output_dir="./model_output",
        deploy="local",  # Change to "cloud" if deploying to GCP
        bucket_name="your-bucket-name",  # Needed for cloud
        job_name="housing_model_training",  # Needed for cloud
        units_per_layer=[5, 10, 15]  # Default unit configuration for layers
    )
    
    # Now call the main function with all the arguments, including units_per_layer
    main(args.output_dir, args.deploy, args.units_per_layer, args.bucket_name, args.job_name)

    # Run TensorBoard in the notebook after training
    %load_ext tensorboard
    %tensorboard --logdir=./model_output/logs


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Number of devices: 1
Epoch 1/30


2025-02-26 12:26:35.775996: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


41/41 [==============================] - 5s 37ms/step - loss: 670254528.0000 - mape: 100.0227 - val_loss: 282923200.0000 - val_mape: 99.8221
Epoch 2/30
41/41 [==============================] - 0s 6ms/step - loss: 668311360.0000 - mape: 99.8035 - val_loss: 278664992.0000 - val_mape: 98.8319
Epoch 3/30
41/41 [==============================] - 0s 6ms/step - loss: 662749248.0000 - mape: 99.1848 - val_loss: 267451616.0000 - val_mape: 96.1875
Epoch 4/30
41/41 [==============================] - 0s 6ms/step - loss: 648774848.0000 - mape: 97.5663 - val_loss: 243336768.0000 - val_mape: 90.2615
Epoch 5/30
41/41 [==============================] - 0s 6ms/step - loss: 620617472.0000 - mape: 94.1373 - val_loss: 200275728.0000 - val_mape: 78.7106
Epoch 6/30
41/41 [==============================] - 0s 6ms/step - loss: 569589696.0000 - mape: 87.6206 - val_loss: 136216752.0000 - val_mape: 58.6658
Epoch 7/30
41/41 [==============================] - 0s 6ms/step - loss: 489513568.0000 - mape: 77.8333 - val_

Reusing TensorBoard on port 6006 (pid 16545), started 0:00:26 ago. (Use '!kill 16545' to kill it.)

1. TensorFlow Serving for Batch Predictions:
To handle large-scale predictions efficiently, I would deploy the model using TensorFlow Serving, which supports batch processing. By serving predictions in batches, I can improve throughput and reduce the overhead of making multiple individual prediction requests. This would allow me to scale predictions in a more cost-effective way, especially when there is a high volume of requests.

2. Cloud Function for Predictions:
Another way to scale predictions would be to deploy the model as a Google Cloud Function or Cloud Run service. Both services are serverless and scale automatically based on incoming traffic. This would be ideal for handling varying prediction loads without worrying about managing infrastructure.

3. Asynchronous Predictions (Queueing):
To improve scalability further, I could use a queueing system like Google Cloud Pub/Sub or Cloud Tasks to manage incoming prediction requests. By asynchronously queuing predictions and processing them in batches, I can decouple the prediction process and ensure that the system can handle high volumes of requests efficiently.

4. Horizontal Scaling:
For even larger prediction loads, I would consider horizontal scaling, where I run multiple instances of my model across different machines or VMs. This could be done using Google Kubernetes Engine (GKE) or Google Cloud Compute Engine, allowing the system to automatically scale out based on the number of requests or resource utilization.

By implementing these strategies, I would ensure that the prediction system can scale effectively to handle increased traffic and workloads.